In [1]:
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()
%matplotlib inline

import keras
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Multiply, Add
from keras.optimizers import Adam, Nadam
import salty
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from random import shuffle
import pandas as pd
import random

#Keras build
from keras import backend as K
from keras.objectives import binary_crossentropy #objs or losses
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Layer
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#cation data
cations = pd.read_csv('../data/cations.csv')
cations = cations['smiles_string']
salts = pd.read_csv('../data/salts.csv')
salts = salts['smiles_string']
categories = pd.read_csv('../data/categories.csv')
categories = categories['category']
coldic = pd.read_csv('../data/coldic.csv')
coldic = coldic.to_dict(orient='records')[0]
salt_coldic = pd.read_csv('../data/salt_coldic.csv')
salt_coldic = salt_coldic.to_dict(orient='records')[0]
salt_categories = pd.read_csv('../data/salt_categories.csv')
salt_categories = salt_categories['category']
density_coldic = pd.read_csv('../data/density_coldic.csv')
density_coldic = density_coldic.to_dict(orient='records')[0]
density_categories = pd.read_csv('../data/density_categories.csv')
density_categories = density_categories['category']

#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *

#training array info
smile_max_length = 105
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

/home/wesleybeckner/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
#plot
import matplotlib.pylab as plt
import seaborn as sns; sns.set()
from matplotlib import colors
from itertools import cycle

#data
import pandas as pd
import numpy as np
from numpy.linalg import norm
import json
import random
import copy

#ML
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as K
from keras import objectives
from keras.objectives import binary_crossentropy #objs or losses
from keras.layers import Dense, Dropout, Input, Multiply, Add, Lambda
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#chem
import salty
from rdkit import Chem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import Draw


class MoleculeVAE():

    autoencoder = None
    
    def create(self,
               charset,
               max_length = 105,
               latent_rep_size = 292,
               weights_file = None,
               qspr = False,
               conv_layers=3,
               gru_layers=3,
               conv_filters=9, 
               conv_kernel_size=9,
               gru_output_units=501):
        charset_length = len(charset)
    
        x = Input(shape=(max_length, charset_length))
        _, z = self._buildEncoder(x, 
                                  latent_rep_size, 
                                  max_length,
                                  conv_layers, 
                                  conv_filters, 
                                  conv_kernel_size)
        self.encoder = Model(x, z)
        encoded_input = Input(shape=(latent_rep_size,))
        self.decoder = Model(
            encoded_input,
            self._buildDecoder(
                encoded_input,
                latent_rep_size,
                max_length,
                charset_length,
                gru_layers,
                gru_output_units
            )
        )

        x1 = Input(shape=(max_length, charset_length))
        vae_loss, z1 = self._buildEncoder(x1, 
                                          latent_rep_size, 
                                          max_length,
                                          conv_layers, 
                                          conv_filters,
                                          conv_kernel_size)
        
        if qspr:
            self.autoencoder = Model(
                x1,
                self._buildDecoderQSPR(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length,
                    gru_layers,
                    gru_output_units
                )
            )

        else:
            self.autoencoder = Model(
                x1,
                self._buildDecoder(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length,
                    gru_layers,
                    gru_output_units
                )
            )
            
        self.qspr = Model(
            x1,
            self._buildQSPR(
                z1,
                latent_rep_size,
                max_length,
                charset_length
            )
        )


        if weights_file:
            self.autoencoder.load_weights(weights_file, by_name = True)
            self.encoder.load_weights(weights_file, by_name = True)
            self.decoder.load_weights(weights_file, by_name = True)
            self.qspr.load_weights(weights_file, by_name = True)
        if qspr:
            self.autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss, 'qspr': 'mean_squared_error'},
                                     metrics = ['accuracy', 'mse'])
        else:
            self.autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss},
                                     metrics = ['accuracy'])
    def _buildEncoder(self, x, latent_rep_size, max_length, epsilon_std = 0.01, conv_layers=3, conv_filters=9, conv_kernel_size=9):
        h = Convolution1D(9, 9, activation = 'relu', name='conv_1')(x)
        for convolution in range(conv_layers-2):
            h = Convolution1D(conv_filters, conv_kernel_size, activation = 'relu', name='conv_{}'.format(convolution+2))(h)
        h = Convolution1D(10, 11, activation = 'relu', name='conv_{}'.format(conv_layers))(h)
        h = Flatten(name='flatten_1')(h)
        h = Dense(435, activation = 'relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev = epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation = 'linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation = 'linear')(h)

        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _buildDecoderQSPR(self, z, latent_rep_size, max_length, charset_length, gru_layers=3):

        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='gru_1')(h)
        for gru in range(gru_layers-2):
            h = GRU(501, return_sequences = True, name='gru_{}'.format(gru+2))(h)
        h = GRU(501, return_sequences = True, name='gru_{}'.format(gru_layers))(h)
        smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='decoded_mean')(h)

        h = Dense(latent_rep_size, name='qspr_input', activation='relu')(z)
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        smiles_qspr = Dense(1, activation='linear', name='qspr')(h)

        return smiles_decoded, smiles_qspr

    def _buildDecoder(self, z, latent_rep_size, max_length, charset_length, gru_layers=3, gru_output_units=501):

        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='gru_1')(h)
        for gru in range(gru_layers-2):
            h = GRU(gru_output_units, return_sequences = True, name='gru_{}'.format(gru+2))(h)
        h = GRU(501, return_sequences = True, name='gru_{}'.format(gru_layers))(h)
        smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='decoded_mean')(h)

        return smiles_decoded
    
    def _buildQSPR(self, z, latent_rep_size, max_length, charset_length):
        h = Dense(latent_rep_size, name='latent_input', activation='relu')(z)
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        return Dense(1, activation='linear', name='qspr')(h)

    def save(self, filename):
        self.autoencoder.save_weights(filename)
    
    def load(self, charset, weights_file, latent_rep_size = 292):
        self.create(charset, weights_file = weights_file, latent_rep_size = latent_rep_size)

In [14]:
vae = MoleculeVAE()
vae.create?

Signature: vae.create(charset, max_length=105, latent_rep_size=292, weights_file=None, qspr=False)
Docstring: <no docstring>
File:      /media/wesleybeckner/weshhd/wes/Dropbox/Python/py3/generative_learners_salts/scripts/vae.py
Type:      method


In [8]:
#training array info
import json
f = open("../data/gdb_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

In [12]:
from scripts import *

In [15]:
vae.create(char_set, 
           max_length=51, 
           latent_rep_size=292,
           weights_file='1mil_GDB17_10.h5',
           qspr=False)

In [ ]:
#For loop to save semi-trained model to
#view PCAs and Z distributions during training

#training array info
smile_max_length = 51
import json
f = open("../data/1mil_GDB17.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training data
chemvae = MoleculeVAE()
char_set = set(char_to_index.keys())
weights_file = '1mil_GDB17_10.h5'
chemvae.load(char_set, weights_file)
# df = pd.read_csv('../data/GDB17.1000000', names=['smiles'])
data_size = 100000


for p in range(0,5):
#     values = df['smiles'][data_size*p:data_size*(p+1)]
#     padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    # for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
#     for i, smile in enumerate(padded_smiles[:data_size]):
    for i in range(data_size):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.33, random_state=42)   
    chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    chemvae.save('1mil_GDB17_500K_mix_500K_cation_{}.h5'.format(p+1))
    

In [3]:
ls *GDB*

1mil_GDB17_10.h5
1mil_GDB17_1.h5
1mil_GDB17_2.h5
1mil_GDB17_3.h5
1mil_GDB17_4.h5
1mil_GDB17_500K_mix_1.h5
1mil_GDB17_500K_mix_2.h5
1mil_GDB17_500K_mix_3.h5
1mil_GDB17_500K_mix_4.h5
1mil_GDB17_500K_mix_500K_cation_1.h5
1mil_GDB17_500K_mix_500K_cation_2.h5
1mil_GDB17_500K_mix_500K_cation_3.h5
1mil_GDB17_500K_mix_500K_cation_4.h5
1mil_GDB17_500K_mix_500K_cation_5.h5
1mil_GDB17_500K_mix_5.h5
1mil_GDB17_5.h5
1mil_GDB17_6.h5
1mil_GDB17_7.h5
1mil_GDB17_8.h5
1mil_GDB17_9.h5
2mol_1mil_GDB17_10.h5
2mol_1mil_GDB17_1.h5
2mol_1mil_GDB17_2.h5
2mol_1mil_GDB17_3.h5
2mol_1mil_GDB17_4.h5
2mol_1mil_GDB17_5.h5
2mol_1mil_GDB17_6.h5
2mol_1mil_GDB17_7.h5
2mol_1mil_GDB17_8.h5
2mol_1mil_GDB17_9.h5
2mol_50mil_GDB17_2.h5
2mol_50mil_GDB17_3_1mil_mix_1.h5
2mol_50mil_GDB17_3_1mil_mix_1mil_cat_1.h5
2mol_50mil_GDB17_3.h5
gen2_2mol_1mil_GDB17_10.h5
gen2_2mol_1mil_GDB17_1.h5
gen2_2mol_1mil_GDB17_2.h5
gen2_2mol_1mil_GDB17_3.h5
gen2_2mol_1mil_GDB17_4.h5
gen2_2mol_1mil_GDB17_5.h5
gen2_2mol_1mil_GDB17_6.h5
gen2_2mol_1mil_G

In [10]:
vae.autoencoder.layers[1].name = 'conv_wes'

In [12]:
vae.autoencoder.layers[1].name

'conv_wes'